In [ ]:
pip install bs4

In [ ]:
pip install wget

# BIBLIOTECAS

In [ ]:
import urllib3
import wget
from bs4 import BeautifulSoup
from urllib.error import *
import warnings
import requests
import os
import gzip
import shutil
from google.cloud import storage
storage_client = storage.Client()

warnings.filterwarnings('ignore')

# FUNÇÕES

In [ ]:
# Função criada para fazer o scrap no site kaggle e retornar os links de armazenamento


def scrap(url):
  try:
    conexao = urllib3.PoolManager() #PoolManager possuir recursos para manipular websites.
    retorno = conexao.request('GET',url) #Realiza a requisição no site e armazena o conteúdo.
    retorno.data
    pagina=BeautifulSoup(retorno.data,"html.parser") #Manipula os dados HTML e organiza a estrutura dos dados.
    print('Leitura realizada com sucesso')
    print(pagina)
    
  except (URLError, HTTPError) as site:
    print('Não foi possivel ler o site', site)

In [ ]:
# Função criada para fazer o scrap no site archive e retornar os links 'href'

conteudo = [] #Lista onde iremos armazenar nossos links

def scrap_archive(url):
  try:
    conexao = urllib3.PoolManager() #PoolManager possuir recursos para manipular websites.
    retorno = conexao.request('GET',url) #Realiza a requisição no site e armazena o conteúdo.
    retorno.data
    pagina=BeautifulSoup(retorno.data,"html.parser") #Manipula os dados HTML e organiza a estrutura dos dados.
    print('Leitura realizada com sucesso')
    for link in pagina.find_all('a'): #Realiza uma varredura no conteúdo e identifica apenas os links 'href'
        conteudo.append(link.get('href')) #Armazena todos os links na nossa lista

  except (URLError, HTTPError) as site:
    print('Não foi possivel ler o site', site)

In [ ]:
# Envio dos datasets para camada landing

def upload_dataset(bucket_name, source_file_name, destination_blob_name):
    '''Upload dos aquivos no datalake'''
   
    # bucket_name = "nome-da-bucket"
    # source_file_name =  caminho do arquivo a ser enviado
    # destination_blob_name = "nome-objeto-bucket"

    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"Arquivo {source_file_name} enviado para {destination_blob_name}."
    )

# NETFLIX

In [ ]:
#Url para função realizar o scrap no kaggle
scrap('https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data')

In [ ]:
#O dataset se encontra armazenad no site'archive' ,Url para a função realizar o scrap no archive
scrap_archive('https://archive.org/details/nf_prize_dataset.tar')

Leitura realizada com sucesso


In [ ]:
conteudo

['#maincontent',
 'https://archive.org',
 'https://archive.org/create/',
 None,
 'https://archive.org/account/signup',
 'https://archive.org/account/login',
 'https://archive.org/web/',
 'https://archive.org/details/texts',
 'https://archive.org/details/movies',
 'https://archive.org/details/audio',
 'https://archive.org/details/software',
 'https://archive.org/details/image',
 'https://archive.org/donate/',
 'https://archive.org/about/',
 'https://archive.org/about/',
 'https://blog.archive.org/',
 'https://archive.org/projects/',
 'https://archive.org/about/faqs.php',
 'https://archive.org/donate/',
 'https://archive.org/about/contact.php',
 'https://archive.org/about/jobs.php',
 'https://archive.org/about/volunteerpositions.php',
 'https://archive.org/about/bios.php',
 'https://archive.org/advancedsearch.php',
 'https://archive.org/account/signup',
 'https://archive.org/account/login',
 '/download/nf_prize_dataset.tar',
 'https://twitter.com/intent/tweet?url=https://archive.org/deta

In [ ]:
# O link para o dataset se encontra na posição 43
site='https://archive.org'
url = site + conteudo[43]
url

'https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz'

In [ ]:
#armazenamento dos datasets
path=('/home/dataproc')

In [ ]:
#Realizando o download do dataset netflix, para o desenvolver do trabalho foi utilizado o arquivo disponibilizado no kaggle,
# pois a única alteração realizada no arquivo original foi a união dos 17.000 arquivos de texto em quatro arquivos de texto.
url_kaggle='https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data/download'
arquivo_local=wget.download(url_kaggle,path)
print('Download finalizado')    

Download finalizado


# AMAZON

In [ ]:
#Url para a função realizar o scrap no arquivo que contém o link para os datasets amazon.
scrap('https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt')

Leitura realizada com sucesso
SAMPLE CONTENT:
https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv
https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_fr.tsv

DATA COLUMNS:
marketplace       - 2 letter country code of the marketplace where the review was written.
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.
review_id         - The unique ID of the review.
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.
product_parent    - Random identifier that can be used to aggregate reviews for the same product.
product_title     - Title of the product.
product_category  - Broad product category that can be used to group reviews 
                    (also used to group the dataset into coherent parts).
star_rating       - The 1-5 star rating of the review.
helpful_votes  

In [ ]:
# path com os datasets relevantes para a nossa análise
url_aws1 = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz'
url_aws2 = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz'
url_aws3 = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'

In [ ]:
#Realizando o download dos datasets amazon.
links = [url_aws1,url_aws2,url_aws3]

for i in links:
    wget.download(i,path)
    print('Download finalizado')


Download finalizado
Download finalizado
Download finalizado


# DATALAKE

In [ ]:
# Envio dos arquivos para camada Landing do datalake

upload_dataset('datalake-5gflix',arquivo_local,'landing/nf_prize_dataset.tar.gz')   

Arquivo /home/dataproc/nf_prize_dataset.tar.gz enviado para landing/nf_prize_dataset.tar.gz.


In [ ]:
upload_dataset('datalake-5gflix',path+'/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz','landing/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz')   

Arquivo /home/dataproc/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz enviado para landing/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz.


In [ ]:
upload_dataset('datalake-5gflix',path+'/amazon_reviews_us_Video_DVD_v1_00.tsv.gz','landing/amazon_reviews_us_Video_DVD_v1_00.tsv.gz')   

Arquivo /home/dataproc/amazon_reviews_us_Video_DVD_v1_00.tsv.gz enviado para landing/amazon_reviews_us_Video_DVD_v1_00.tsv.gz.


In [ ]:
upload_dataset('datalake-5gflix',path+'/amazon_reviews_us_Video_v1_00.tsv.gz','landing/amazon_reviews_us_Video_v1_00.tsv.gz')   

Arquivo /home/dataproc/amazon_reviews_us_Video_v1_00.tsv.gz enviado para landing/amazon_reviews_us_Video_v1_00.tsv.gz.
